<a href="https://colab.research.google.com/github/Bellajia721/WeatherWise_A2/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install requests
!pip install matplotlib
!pip install pyinputplus

In [2]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: 3928442b4d494e88b28a9f51e517c5d9.1iDEI4NQu_Lf6p2J_pnZHwur


## 📦 Setup and Configuration
Import required packages and setup environment.

In [3]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here
from ipywidgets import Text, DatePicker, Button, VBox
from IPython.display import display

city_input = Text(description = 'City:', placeholder = 'e.g. Perth')
date_input = DatePicker(description = 'Date:')
submit_button = Button(description='Get Weather', button_style='success')

def on_submit_button_click(b):
    city = city_input.value
    date = date_input.value.strftime('%m-%d-%y')
    print(f'City: {city}')
    print(f'Date: {date}')

submit_button.on_click(on_submit_button_click)

display(VBox([city_input, date_input, submit_button]))

City: Perth
Date: 10-13-25


## 🌤️ Weather Data Functions

In [20]:
# Define get_weather_data() function here
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    try:
      raw = get_weather(location = location, format="raw_json", use_mock = False)
      if isinstance(raw, str) and raw.startswith("Error:"):
        print(f"Weather API returned an error response: {raw}")
        return None
    except Exception as e:
      print(f"An error occurred: {e}")
      return None

    current_weather = (raw.get("current_condition") or [{}])[0]
    desc = None
    if current_weather.get("weatherDesc"):
      desc = (current_weather["weatherDesc"][0] or {}).get("value")
    forecast_weather = (raw.get("weather") or [])[:forecast_days]

    return {
        "location": location,
        "current_weather": {
            "temperature": current_weather.get("temp_C"),
            "windspeed": current_weather.get("windspeedKmph"),
            "humidity": current_weather.get("humidity"),
            "description": desc
        },
        "daily":[
            {
                 "date": d.get("date"),
                 "average_temp": d.get("avgtempC"),
                 "total_precip": d.get("totalprecipMM"),
                 "max_temp": d.get("maxtempC"),
                 "min_temp": d.get("mintempC"),
        }
        for d in forecast_weather
      ]
    }

## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass


In [ ]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass

## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.